In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from committee_learning.initial_conditions import SymmetricInitialConditions
from committee_learning.simulation import Simulation
from committee_learning.ode import SquaredActivationODE
from committee_learning.result import SimulationResult, SquareODEResult
from committee_learning.ode.epsilon0 import EpsilonZeroODE

import numpy as np

# d_list = [100,1000,10000]#,20000]
d_list = [10000]
p = 8
k = 4
gamma0 = 1.
noise = 1e-1
dt = 1e-3
epsilon = 0.
q0 = 2.

analytic = EpsilonZeroODE(p,k,gamma0,noise,q0)

print(f'Actual initial value:\t{analytic.q0}')
print(f'Diverging threeshold:\t{analytic.qf2}')
print(f'Attractor:\t\t{analytic.qf1}')
print(f'Risk minimum q:\t\t{analytic.qf0}')

ic = SymmetricInitialConditions(p,k,epsilon,q0)

Actual initial value:	2.0
Diverging threeshold:	5.78100796675811
Attractor:		0.8856586999085575
Risk minimum q:		0.8


### Simulations

In [4]:
simrs = {}
for d, Wt, W0 in ic.simulations(d_list):
  print(f'Simulating d = {d}')
  sim = Simulation(d,p,k,noise,Wt,gamma0,'square',W0)
  log_lenght = 2+np.log10(d)
  sim_res = SimulationResult(f'symmetric_eps={epsilon:.5e}')
  sim_res.from_file_or_run(sim, log_lenght,path='computation-database/sim/')
  simrs[d] = sim_res

Simulating d = 10000


### ODE

In [5]:
ode = SquaredActivationODE(p,k,noise,gamma0,ic.P,ic.Q,ic.M,dt)
oder = SquareODEResult(f'symmetric_eps={epsilon:.5e}',ode=ode)

ode.fit_logscale(2, 100)

100%|██████████| 90000/90000 [00:12<00:00, 7039.43it/s]


In [10]:
import matplotlib.pyplot as plt
from committee_learning.utilities import plot_style

with plot_style():
  fig, ax = plt.subplots(figsize=(2.75,2.75))
  ax.set_xscale('log')
  ax.set_yscale('log')
  ax.set_xlabel(r'$t$')
  ax.set_ylabel(r'$\mathcal{R}$')
  ax.set_xlim(min(ode.saved_times), max(ode.saved_times))
  ax.set_ylim(1e-2,1.5) # Global plot

  # Lines
  ax.axhline(y=analytic.R0,ls='-.',lw=0.7,label=r'$\mathcal{R}{(0)}$',color='orange')
  ax.axhline(y=analytic.R_minimum,ls='-.',lw=0.7,label=r'theoretical min. of $\mathcal{R}$',color='blue')
  ax.axhline(y=analytic.R_qf1,ls='-.',lw=0.7,label=r'$\mathcal{R}{(q_f^1)}$',color='green')
  # ax.axhline(y=R_qf2,ls='-.',lw=0.7,label=r'$\mathcal{R}{(q_f^2)}$',color='red')


  # Plot simulations
  for d, sr in simrs.items():
    t = np.array(sr.steps) / (d)
    ax.plot(t, sr.macroscopic_risk(), label=f'Sim.',ls='--',lw=1.,marker='o',ms=5., color='m')

  # Plot ODE
  ax.plot(ode.saved_times, ode.saved_risks, label='ODE')

  ax.legend()
  fig.savefig('figures/example-eps0.pdf', bbox_inches = 'tight')
  ax.set_ylim(1e-1,1.5) # Zoomed
  fig.savefig('figures/example-eps0-zoomed.pdf', bbox_inches = 'tight')
  